In [1]:
#from google.colab import output
import geemap

In [2]:
import ee
ee.Authenticate()
ee.Initialize(project='ee-anaad')

In [3]:
import pandas as pd
pd.set_option('display.max_columns', None)
files = ["Raman/climate_data_"+str(i)+".csv" for i in range(660)]
df = pd.concat([pd.read_csv(f, index_col=0) for f in files])
print("Complete Data", len(df)) #Net Data
#Drop NA rows
df = df.dropna().reset_index(drop=True)
print("Data after droping Na", len(df))

Complete Data 68078
Data after droping Na 62319


In [4]:
district_boundaries = ee.FeatureCollection('projects/ee-anaad/assets/India_district_boundaries')
aez_boundaries = ee.FeatureCollection('projects/ee-anaad/assets/India_AEZ_boundaries')
block_boundaries = ee.FeatureCollection('projects/ee-anaad/assets/AEZ_mapped_block_boundaries')
blocks = block_boundaries.filter(ee.Filter.eq('aez_id', 12))
blocks_list = blocks.toList(blocks.size().getInfo())
info = blocks_list.map(lambda x: ee.Feature(x).toDictionary())
block_df = pd.DataFrame.from_records(info.getInfo())
block_df = block_df[["District N", "District c", "Subdistric", "Subdistr_1"]]
copy_groupby = block_df.groupby(["District N",  "Subdistric"]).size().reset_index().rename(columns={0:"count"})
copies = copy_groupby[copy_groupby["count"]>1].reset_index(drop=True)
copies

,District N,Subdistric,count
0,balangir,Bangomunda,2
1,cuttack,Choudwar,2
2,cuttack,Kanpur,2
3,dakshin bastar dantewada,Chhindgarh,2
4,ganjam,SERAGAD,2
5,malkangiri,Chitrakonda,2
6,vizianagaram,Balijipeta,2


In [5]:
#from google.colab import output
import geemap
#from google.colab import output
#output.enable_custom_widget_manager()
# Run the code below to visualize AEZ boundaries
m = geemap.Map()
m.addLayerControl()
names = blocks_list.map(lambda x: ee.Feature(x).get("Subdistric")).getInfo()

for i in range(10):
    print(i)
    block = ee.Feature(blocks_list.get(i))
    m.addLayer(block, {}, 'Blocks_' + names[i])

0
1
2
3
4
5
6
7
8
9


In [6]:
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [7]:
block = ee.Feature(blocks_list.get(0))

In [8]:
block.geometry().area().getInfo()/1e6

451.25986624981704

In [17]:
image = ee.ImageCollection("NASA/GLDAS/V021/NOAH/G025/T3H")\
        .filterBounds(block.geometry())\
        .filterDate('2020-06-01','2020-06-02').first()

In [20]:
image = image.clip(block.geometry())

In [29]:
m.addLayer(image, {"bands":["CanopInt_inst",]}, 'image1')
m

Map(bottom=112755.0, center=[24.633286469281973, 86.0881805419922], controls=(WidgetControl(options=['position…

In [24]:
image